In [82]:
from gensim.models import Word2Vec, KeyedVectors
vecmodel = KeyedVectors.load_word2vec_format('../../../GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000)


In [83]:

from __future__ import print_function

import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Convolution2D, MaxPooling2D
from keras.models import Model
from keras.layers import Embedding


In [93]:
import pandas as pd
columns = "['pair_ID', 'sentence_A', 'sentence_B', 'relatedness_score','entailment_judgment']"
train_df = pd.read_csv("../data/SemEval2014_dataset/SICK_train.txt", sep='\t')
trial_df = pd.read_csv("../data/SemEval2014_dataset/SICK_trial.txt", sep='\t')
test_df = pd.read_csv("../data/SemEval2014_dataset/SICK_test_annotated.txt", sep='\t')

texts = []
BASE_DIR = ''
TEXT_DATA_DIR = os.path.join('../data/SemEval2014_dataset/')
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
# VALIDATION_SPLIT = 0.2

In [94]:
# create word_id
merged = train_df['sentence_A'].tolist()
merged.extend(train_df['sentence_B'].tolist())
merged.extend(trial_df['sentence_A'].tolist())
merged.extend(trial_df['sentence_B'].tolist())
merged.extend(test_df['sentence_A'].tolist())
merged.extend(test_df['sentence_B'].tolist())

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(merged)
sequences = tokenizer.texts_to_sequences(merged)


In [95]:
embeddings_index = {}
word_index = tokenizer.word_index
# prepare embedding matrix
num_words = max(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if word in vecmodel.vocab:
        embedding_vector = vecmodel[word]
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [96]:
def get_tokenized_padded_data(data):
    
#     t = Tokenizer(num_words=MAX1_NB_WORDS)
#     t.fit_on_texts(data)
    seq = tokenizer.texts_to_sequences(data)
    d = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)   
    return d
def get_target_category(data):
    target= np.zeros([len(data),6])
    for i in range(len(data)):
        value = int(round(data[i]))
        target[i][value] = 1
    return target

In [97]:

xA_train = get_tokenized_padded_data(train_df['sentence_A'].tolist())
xA_val = get_tokenized_padded_data(trial_df['sentence_A'].tolist())
xA_test = get_tokenized_padded_data(test_df['sentence_A'].tolist())

xB_train = get_tokenized_padded_data(train_df['sentence_B'].tolist())
xB_val = get_tokenized_padded_data(trial_df['sentence_B'].tolist())
xB_test = get_tokenized_padded_data(test_df['sentence_B'].tolist())

y_train = get_target_category(train_df['relatedness_score'].tolist())
y_val = get_target_category(trial_df['relatedness_score'].tolist())
y_test = get_target_category(test_df['relatedness_score'].tolist())


In [98]:
xA_train.shape

(4500, 30)

In [35]:
# inp = Input(shape=(MAX_SEQUENCE_LENGTH,))

In [36]:
# embedding_layer = Embedding(num_words + 1,
#                             EMBEDDING_DIM,
#                             weights=[embedding_matrix],
#                             input_length=MAX_SEQUENCE_LENGTH,
#                             trainable=False)

In [37]:
# seqA_input = Input(shape=(MAX_SEQUENCE_LENGTH,))

# embedding_layer = Embedding(num_words,
#                             EMBEDDING_DIM,
#                             weights=[embedding_matrix],
#                             input_length=MAX_SEQUENCE_LENGTH,
#                             trainable=False)

# print (seqA_input.shape)
# x_A = embedding_layer(seqA_input)
# print (x_A.shape)
# x_A = Convolution2D(filters=300,
#                          kernel_size=[ 1 , 1 ],
#                          padding="valid",
#                          activation="relu",
#                          strides=1)(x_A)

# print (x_A.shape)
# x_A = MaxPooling2D(pool_size=[MAX_SEQUENCE_LENGTH,1])(x_A)

# # x_B = embedding_layer(sequence_input)
# # x_B = Convolution2D(filters=300,
# #                          kernel_size=[ 1 , 1 ],
# #                          padding="valid",
# #                          activation="relu",
# #                          strides=1)(x_A)

# # x_B = MaxPooling2D(pool_size=[MAX_SEQUENCE_LENGTH,1])(x_B)


# x_A = Dense(300, activation='relu')(x_A)
# preds = Dense(5, activation='softmax')(x_A)
# model = Model(seqA_input, preds)
# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['acc'])
# model.summary()
# # model.fit(xA_train, y_train,
# #           batch_size=128,
# #           epochs=10,
# #           validation_data=(xA_val, y_val))

In [105]:
seqA_input = Input(shape=(MAX_SEQUENCE_LENGTH,), name='seqA_input')
seqB_input = Input(shape=(MAX_SEQUENCE_LENGTH,))

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print (seqA_input.shape)
x_A = embedding_layer(seqA_input)
x_A = Conv1D(filters=300,kernel_size=1,padding="valid",activation="relu",strides=1,kernel_initializer='he_uniform')(x_A)
x_A = MaxPooling1D(pool_size=[MAX_SEQUENCE_LENGTH])(x_A)

x_B = embedding_layer(seqB_input)
x_B = Conv1D(filters=300,kernel_size=1,padding="valid",activation="relu",strides=1,kernel_initializer='he_uniform')(x_B)
x_B = MaxPooling1D(pool_size=[MAX_SEQUENCE_LENGTH])(x_B)

diff = keras.layers.Subtract()([x_A, x_B])
prod = keras.layers.Multiply()([x_A, x_B])


x = keras.layers.concatenate([diff, prod])


x = Dense(300, activation='tanh',kernel_initializer='he_uniform')(x)
x = GlobalMaxPooling1D()(x)
preds = Dense(6, activation='softmax')(x)
model = Model(inputs=[seqA_input,seqB_input], outputs=preds)
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['acc'])


(?, 30)


In [106]:
model.fit([xA_train,xB_train], y_train,
          batch_size=128,
          epochs=10,validation_data=([xA_val,xB_val], y_val))

Train on 4500 samples, validate on 500 samples
Epoch 1/10
4500/4500 [==============================] - 4s 813us/step - loss: 0.1226 - acc: 0.3740 - val_loss: 0.1238 - val_acc: 0.3700
Epoch 2/10
4500/4500 [==============================] - 3s 739us/step - loss: 0.1110 - acc: 0.4653 - val_loss: 0.1222 - val_acc: 0.3520
Epoch 3/10
4500/4500 [==============================] - 3s 753us/step - loss: 0.1020 - acc: 0.5244 - val_loss: 0.1270 - val_acc: 0.3540
Epoch 4/10
4500/4500 [==============================] - 4s 810us/step - loss: 0.0945 - acc: 0.5671 - val_loss: 0.1264 - val_acc: 0.3580
Epoch 5/10
4500/4500 [==============================] - 4s 788us/step - loss: 0.0852 - acc: 0.6309 - val_loss: 0.1323 - val_acc: 0.3580
Epoch 6/10
4500/4500 [==============================] - 4s 793us/step - loss: 0.0764 - acc: 0.6773 - val_loss: 0.1407 - val_acc: 0.3440
Epoch 7/10
4500/4500 [==============================] - 4s 787us/step - loss: 0.0702 - acc: 0.7087 - val_loss: 0.1451 - val_acc: 0.3680
E

In [92]:
model.evaluate([xA_test,xB_test], y_test, batch_size=128)

4927/4927 [==============================] - 3s 538us/step


[0.14651936665252954, 0.34442865850658333]

In [ ]:
np.reshape(y_train,[-1,1])

In [ ]:
a = np.array([1, 2, 3, 4])
b = np.array([11, 12, 33, 54])
np.concatenate((a,b))